In [ ]:
import os
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
from statsmodels.graphics.factorplots import interaction_plot
import numpy as np
from sklearn.metrics import cohen_kappa_score
from scipy import stats

In [ ]:
def compute_anova(df, dependent_v):
    """

    Args:
        df: data frame that contains the data
        dependent_v: dependent variable

    Returns:

    """
    aov = pg.mixed_anova(dv=dependent_v, within='mode', between='condition', subject='id', data=df)
    # printing of ANOVA summary
    pg.print_table(aov)

In [ ]:
file_velocities = input()
#/home/yesid/Documents/Master_semester3/VR/postural_stability_analysis/data/all_velocities_.csv
df_velocities = pd.read_csv(file_velocities)
df_velocities = df_velocities[['id', 'condition', 'mode', 'condition_mode', 'station',
       'time_frame_begin', 'time_frame_end', 'motion_sickness_score',
       'name_of_audio_data', 'motionsickness_score_rating_begin',
       'motionsickness_score_rating_accepted_timeStamp', 'average_velocity']]

In [ ]:
def check_assumptions(df, dv, dv_name):
    '''
    Checks the assumptions of a mixed ANOVA.

    df (pd.DataFrame)
    dv (str): dependent variable column name
    dv_name (str): name of the dependent variable (to be printed)
    '''

    print("Checking the assumptions for the dependent variable {}...\n".format(dv_name))
    all_assumptions = []

    # Assumption of the residuals being normally distributed
    df['factor_comb'] = df["condition"] + '-'+ df["mode"]
    normal = pg.normality(data=df, dv=dv, group='factor_comb')
    if normal["normal"].all():
        print("The residuals are approximately normally distributed for each level of the within-subjects factor (tested using the Shapiro-Wilks Test)")
        all_assumptions.append(True)
    else:
        print("The residuals are NOT approximately normally distributed for each level of the within-subjects factor! (tested using the Shapiro-Wilks Test)")
        all_assumptions.append(False)

    print()
    # Assumption of homoscedasticity
    print("There should be equal variance for every level of within-subject factor:")
    hybrid = df[df["mode"]=="Hybrid"].reset_index(drop=True)
    homo_hybrid = pg.homoscedasticity(data=hybrid, dv=dv, group='condition')
    all_assumptions.append(homo_hybrid["equal_var"][0])
    print("For the level Hybrid, Levene's test resulted in a p-value of {}, indicating that the assumption of equal variances is {}.".format(homo_hybrid["pval"][0].round(3), homo_hybrid["equal_var"][0]))

    fp = df[df["mode"]=="Firstperson"].reset_index(drop=True)
    homo_fp = pg.homoscedasticity(data=fp, dv=dv, group='condition')
    all_assumptions.append(homo_fp["equal_var"][0])
    print("For the level First Person, Levene's test resulted in a p-value of {}, indicating that the assumption of equal variances is {}.".format(homo_fp["pval"][0].round(3), homo_fp["equal_var"][0]))

    print()
    # Assumption of sphericity
    sphericity = pg.sphericity(data=df, dv=dv, subject='id', within='condition')
    all_assumptions.append(sphericity[0])
    print("Mauchly’s test of sphericity resulted in a p-value of {}, indicating that the assumption is {}.".format(sphericity[-1], sphericity[0]))

    print()
    # Assumption of homogeneity of the variance-covariance matrices:
    print("The homogeneity of variance-covariance matrices formed by the between-subject factor for each level of the within-subject factor should be equal:")
    box_fp = pg.box_m(data=fp, dvs=[dv], group='condition', alpha=0.001)
    all_assumptions.append(box_fp["equal_cov"][0])
    print("For the level First Person, Box’s M test resulted in a p-value of {}, indicating that the assumption is {}.".format(box_fp["pval"][0].round(3), box_fp["equal_cov"][0]))

    box_hybrid = pg.box_m(data=hybrid, dvs=[dv], group='condition', alpha=0.001)
    all_assumptions.append(box_hybrid["equal_cov"][0])
    print("For the level Hybrid, Box’s M test resulted in a p-value of {}, indicating that the assumption is {}.".format(box_hybrid["pval"][0].round(3), box_hybrid["equal_cov"][0]))

    print()
    # Assumption of no outliers
    #outliers = df[df[dv] > df[dv].mean() + 3 * df[dv].std()]
    outliers = df[np.abs(df[dv] - df[dv].mean()) > (3 * df[dv].std())]
    print("A data point is called an outlier if it > 3 sd away from the mean.")
    if outliers.empty:
        print("There are no outliers!")
        all_assumptions.append(True)
    else:
        print("There are outliers! The outliers are the participants {}".format(outliers["id"].values))
        print(outliers)
        all_assumptions.append(False)


    if all(x == True for x in all_assumptions):
        print()
        print("ALL ASSUMPTIONS ARE TRUE!")

## Postural Stability Test

In [ ]:
df_velocities.head()

### Visualizations
Before conducting any analysis we will visually explore the data.

#### Data distribution
Below you can find the distribution of the average velocity by condition + mode and, condition alone. As one can easily tell, their distributions do not differ much from each other.

In [ ]:
sns.displot(df_velocities, x="average_velocity", col='condition_mode')
# Divide the figure into a 1x2 grid
sns.displot(df_velocities, x="average_velocity",col ='condition')

The average velocity tends to 0, as it is shown above. On ther hand, if we observe the distribution of the motion sickness scores (below) we can see that it is skewed to the left: most participants said not to have been motion sick. The average velocity, as long as it is a measurement of sickness, seems to back up the motion sickness scores obtained.

In [ ]:
df_velocities['motion_sickness_score'].plot(kind ='hist', title='Motion Sickness Score', figsize=(3,3))

#### Boxplot
As the two boxplot below show, the data contain several outliers.

In [ ]:
sns.set(rc = {'figure.figsize':(11,8)})
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(121)
sns.boxplot(x='condition', y='average_velocity', hue='mode', data=df_velocities, ax=ax1)
ax2 = fig.add_subplot(122)
sns.boxplot(x='station', y='average_velocity', hue='mode', data=df_velocities,ax=ax2)

In [ ]:
sns.pointplot(data=df_velocities, x='mode', y='average_velocity', hue='condition', dodge=True, markers=['o', 's'],
	      capsize=.1, errwidth=1, palette='colorblind')

### Descriptive Statistics

### Mixed ANOVA

#### Description of the analysis

- Between subject variable (factor): Condition (Avatar, Blob) each subject is assigned only once to treatment.
- Within subject: subject assigned both treatments (Hybrid, first person)
- Dependent variable: average_velocity

In [ ]:
# Compute the two-way mixed-design ANOVA
compute_anova(df_velocities, 'average_velocity')

At a significance level 0.05, we can say that there is no significant interaction effect. The effect of the condition/treatment on the average velocity (dependent variable) does not depend on the mode or perspective (Hybrid/Firstperson). Hence, whether someone is motion sick (high velocity) or not (low velocity) does not depend on the perspective used to navigate the VR environment.

#### Assumption check

In [ ]:
check_assumptions(df_velocities,'average_velocity', 'average_velocity')